In [87]:
import pyepo
import torch
from torch import nn
from torch.utils.data import DataLoader
import numpy as np

In [58]:
# model for shortest path
grid = (5,5) # grid size
optmodel = pyepo.model.grb.shortestPathModel(grid)

# generate data
num_data = 100 # number of data
num_feat = 5 # size of feature
deg = 4 # polynomial degree
noise_width = 0.5 # noise width
x, c = pyepo.data.shortestpath.genData(num_data, num_feat, grid, deg, noise_width, seed=135)

In [59]:
from pyepo.data.dataset import optDataset

In [60]:
arcs = []
for i in range(grid[0]):
    # edges on rows
    for j in range(grid[1] - 1):
        v = i * grid[1] + j
        arcs.append((v, v + 1))
    # edges in columns
    if i == grid[0] - 1:
        continue
    for j in range(grid[1]):
        v = i * grid[1] + j
        arcs.append((v, v + grid[1]))

In [61]:
# build dataset
dataset = optDataset(optmodel, x, c)

Optimizing for optDataset...


100%|██████████| 100/100 [00:00<00:00, 1610.52it/s]


In [62]:
# get data loader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [65]:
# build linear model
class LinearRegression(nn.Module):

    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(5, 40)

    def forward(self, x):
        out = self.linear(x)
        return out
# init
predmodel = LinearRegression()

In [67]:

# get data loader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# build linear model
class LinearRegression(nn.Module):

    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(5, 40)

    def forward(self, x):
        out = self.linear(x)
        return out
# init
predmodel = LinearRegression()
# set optimizer
optimizer = torch.optim.Adam(predmodel.parameters(), lr=1e-3)
# init SPO+ loss
spo = pyepo.func.SPOPlus(optmodel, processes=2)

Num of cores: 2


In [80]:
from pyepo import EPO
def regret(predmodel, optmodel, dataloader):
    """
    A function to evaluate model performance with normalized true regret

    Args:
        predmodel (nn): a regression neural network for cost prediction
        optmodel (optModel): an PyEPO optimization model
        dataloader (DataLoader): Torch dataloader from optDataSet

    Returns:
        float: true regret loss
    """
    # evaluate
    predmodel.eval()
    loss = 0
    optsum = 0
    # load data
    for data in dataloader:
        x, c, w, z = data
        # cuda
        if next(predmodel.parameters()).is_cuda:
            x, c, w, z = x.cuda(), c.cuda(), w.cuda(), z.cuda()
        # predict
        with torch.no_grad(): # no grad
            cp = predmodel(x).to("cpu").detach().numpy()
        # solve
        for j in range(cp.shape[0]):
            # accumulate loss
            loss += calRegret(optmodel, cp[j], c[j].to("cpu").detach().numpy(),
                              z[j].item())
        optsum += abs(z).sum().item()
    # turn back train mode
    predmodel.train()
    # normalized
    return loss / (optsum + 1e-7)


def calRegret(optmodel, pred_cost, true_cost, true_obj):
    """
    A function to calculate normalized true regret for a batch

    Args:
        optmodel (optModel): optimization model
        pred_cost (torch.tensor): predicted costs
        true_cost (torch.tensor): true costs
        true_obj (torch.tensor): true optimal objective values

    Returns:predmodel
        float: true regret losses
    """
    # opt sol for pred cost
    optmodel.setObj(pred_cost)
    sol, _ = optmodel.solve()
    # obj with true cost
    obj = np.dot(sol, true_cost)
    # loss
    if optmodel.modelSense == EPO.MINIMIZE:
        loss = obj - true_obj
    if optmodel.modelSense == EPO.MAXIMIZE:
        loss = true_obj - obj
    return loss


In [86]:
# training
num_epochs = 20
for epoch in range(2):
    print("epoch = ",epoch)
    it = 0
    for data in dataloader:
        x, c, w, z = data # x: feature, c: cost, w: 
        # forward pass
        cp = predmodel(x)
        
        # print("it: ", it,"x shape = ",np.shape(x)) 
        # print("w: ", w," w shape = ",np.shape(w)) 
        # print("z: ", z," z shape = ",np.shape(z)) 
        # it = it + 1
        # SPO+ loss
        loss = spo(cp, c, w, z)
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print("x: ", x[0,:]) 
    print("cp: ", cp[0,:]) 
        # regret = pyepo.metric.regret(reg, optmodel, loader_test)
    #     # print("it = ",it,"x = ",spo.optmodel.x)

epoch =  0
x:  tensor([-0.8629, -2.2003, -0.1394,  1.8497,  0.2735])
cp:  tensor([-0.8148,  1.1028,  0.7246, -1.8341, -0.5219,  0.6857, -0.4815,  1.1070,
         0.7286, -0.2700, -0.5083, -1.1259, -0.4710, -0.9342,  0.1993,  0.8728,
        -1.2045, -0.3845,  0.3084,  1.4701,  1.1125,  1.5412, -1.0259, -0.3662,
         0.8940, -0.9008, -0.0992, -0.1185,  0.4493,  1.0897,  0.5000,  0.1889,
        -0.0701, -0.8649, -0.2057, -0.2844,  1.2203,  1.1408, -0.3450, -0.4339],
       grad_fn=<SliceBackward0>)
epoch =  1
x:  tensor([-0.2271,  0.5185, -0.8914,  1.2301, -1.0326])
cp:  tensor([ 0.6273, -0.3287,  0.5408, -0.7008, -0.6884,  0.3619, -0.5723,  1.2862,
         0.7763, -0.5891, -0.2004,  0.4121, -0.5781,  0.2079,  0.5674,  0.1987,
        -0.0914,  0.3493,  0.9369,  0.1259, -0.2955,  0.4816, -0.0677, -0.0361,
         0.9792,  0.1179, -0.0479,  0.3512, -0.3103,  0.1986, -0.3370,  0.6545,
        -0.4693,  0.6527, -0.4147, -1.1494, -0.1759, -0.6625,  0.2150, -0.2998],
       grad_fn=<S

In [79]:
loss.item()

14.679788589477539